In [ ]:
from transformers import AutoModelForCausalLM , AutoTokenizer
import torch

PATH = "/mnt/nvme/MODELS/LLM/glaive-function-calling-v2-small/"

tokenizer = AutoTokenizer.from_pretrained(PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    PATH, 
    trust_remote_code=True,
    device_map="cuda:0",
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True
)

In [ ]:
prompt = """
SYSTEM: You are an helpful assistant who has access to the following functions to help the user, you can use the functions if needed-
[
  {
    "name": "tv.is_on",
    "description": "Check if the TV is on. Returns True if the TV is on, False otherwise",
    "parameters": [
      "name": {
        "type": "string",
        "description": "name of TV to check"
      }
  },
  {
    "name": "tv.get",
    "description": "Get the list of TVs",
    "parameters": [],
    "returns": "list of TVs"
  }
]
USER: Hi, I am trying to watch TV but it is not working.
ASSISTANT: Sure, I can help with that. Do you know the name of the TV?
USER: Yes, its called "Living Room TV"
ASSISTANT: <functioncall> {"name": "tv.is_on", "arguments": '{"name": "Living Room TV"}'}
FUNCTION: {"tv.is_on": None}
"""

inputs = tokenizer(prompt,return_tensors="pt").to(model.device)

# outputs = model.generate(**inputs,do_sample=True,temperature=0.1,top_p=0.95,max_new_tokens=100)
outputs = model.generate(**inputs,do_sample=False,max_new_tokens=512)

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

In [ ]:
prompt = """
SYSTEM: You are an helpful assistant who has access to the following functions to help the user, you can use the functions if needed-
[
    {
        "name": "cab.ride",
        "description": "Find suitable ride for customers given the location, type of ride, and the amount of time the customer is willing to wait as parameters.",
        "parameters":  {
            "start_loc": {
                "type": "string",
                "description": "location of the starting place of the uber ride"
            },
            "end_loc": {
                "type": "string",
                "description": "location of the ending place of the uber ride"
            },
            "type": {
                "type": "string",
                "enum": ["normal", "plus", "premium"],
                "default": "normal",
                "description": "types of uber ride user is ordering"
            },
            "time": {
                "type": "integer",
                "description": "the amount of time in minutes the customer is willing to wait in minutes"
            },
            "platform": {
                "type": "string",
                "default": "uber",
                "enum": ["uber", "ola"],
                "description": "the platform the user is ordering the ride from"
            }
        },
        "required": ["start_loc", "end_loc", "type", "time", "platform"]
    }
]
USER: call an cab to nodia sector 62 in twenty two minutes using ola.
"""

inputs = tokenizer(prompt,return_tensors="pt").to(model.device)

# outputs = model.generate(**inputs,do_sample=True,temperature=0.7,top_p=0.95,max_new_tokens=512)
outputs = model.generate(**inputs,do_sample=False,max_new_tokens=512)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

In [ ]:
len(tokenizer.encode("""{"name": "cab.ride", "arguments": '{"start_loc": "nodia","end_loc": "nodia","type": "normal","time": 120,"platform": "ola"}'}""", return_tensors="pt")[0])